In [1]:
import numpy as np
import boto3
import sagemaker
import io
import sagemaker.amazon.common as smac
import os
import pandas as pd
import sys

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, accuracy_score, f1_score, r2_score
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings('ignore')



from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sagemaker import get_execution_role



get_ipython().system('{sys.executable} -m pip install sagemaker -U')

sagemaker_session = sagemaker.Session()
role = get_execution_role() 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/21/25 02:43:23] INFO     Found credentials from IAM Role:                                   ]8;id=785779;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=869042;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[01/21/25 02:43:28] INFO     generated new fontManager                                         ]8;id=116173;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/matplotlib/font_manager.py\font_manager.py]8;;\:]8;id=321894;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/matplotlib/font_manager.py#1584\1584]8;;\

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 96.1 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.237.1
    Uninstalling sagemaker-2.237.1:
      Successfully uninstalled sagemaker-2.237.1


[01/21/25 02:43:43] INFO     Found credentials from IAM Role:                                   ]8;id=150288;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=558475;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[01/21/25 02:43:44] INFO     Found credentials from IAM Role:                                   ]8;id=670683;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=427925;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [2]:
from io import StringIO
s3 = boto3.resource('s3')
bucket = 'nimbleai-whizlabs'
object_key = 'fraudTrain.csv'


csv_obj = s3.Object(bucket, object_key)
csv_string = csv_obj.get()['Body'].read().decode('utf-8')

fraud_df = pd.read_csv(StringIO(csv_string))

[01/21/25 02:43:52] INFO     Found credentials from IAM Role:                                   ]8;id=136847;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=474228;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [3]:
fraud_df['trans_date_trans_time'] = pd.to_datetime(fraud_df['trans_date_trans_time'])
print(fraud_df.dtypes['trans_date_trans_time'])

datetime64[ns]


In [4]:
fraud_df['trans_hour'] = fraud_df['trans_date_trans_time'].dt.hour
fraud_df['trans_day_of_week'] = fraud_df['trans_date_trans_time'].dt.dayofweek +1
fraud_df['trans_day_of_week'] = fraud_df['trans_day_of_week'].astype(int)

fraud_df['trans_year_month'] = fraud_df['trans_date_trans_time'].dt.to_period('M')

fraud_df['dob']= pd.to_datetime(fraud_df['dob'])
fraud_df['age'] = ((fraud_df['trans_date_trans_time'] - fraud_df['dob']).dt.days/365.25).astype(int)

In [5]:
fraud_df['category'] = fraud_df['category'].astype('category')
fraud_df['gender'] = fraud_df['gender'].astype('category')
fraud_df['is_fraud'] = fraud_df['is_fraud'].astype('category')

In [6]:
fraud_df.shape

(1048575, 27)

In [7]:
custom_bins = [13, 19, 32, 42, 50,62 , float('inf')]
custom_labels = ['Teenagers', 'Young Adults', 'Adults', 'Middle-aged', 'Seniors', 'Retired']

fraud_df['age_category'] = pd.cut(fraud_df['age'], bins=custom_bins, labels=custom_labels, right=False)

print(fraud_df[['age', 'age_category']].tail())

         age  age_category
1048570   76       Retired
1048571   20  Young Adults
1048572   67       Retired
1048573   29  Young Adults
1048574   22  Young Adults


In [8]:
x =  [ 'amt', 'gender','lat', 'long', 'city_pop',  'trans_hour','trans_day_of_week', 'age','is_fraud','age_category']
fraud_df = pd.get_dummies(fraud_df[x], drop_first=True)
fraud_df.columns

Index(['amt', 'lat', 'long', 'city_pop', 'trans_hour', 'trans_day_of_week',
       'age', 'gender_M', 'is_fraud_1', 'age_category_Young Adults',
       'age_category_Adults', 'age_category_Middle-aged',
       'age_category_Seniors', 'age_category_Retired'],
      dtype='object')

In [9]:
fraud_df.shape

(1048575, 14)

In [10]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 119.2 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from xgboost import XGBClassifier

prefix = "fraud-detection-xgboost"

predictors =  [ 'amt',  'city_pop', 'trans_hour', 'age', 'gender_M','lat', 'long','trans_day_of_week','age_category_Young Adults',
       'age_category_Adults', 'age_category_Middle-aged',
       'age_category_Seniors', 'age_category_Retired']

X = fraud_df[predictors]
y = fraud_df[ 'is_fraud_1']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# Calculate scale_pos_weight for handling class imbalance
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

# Define the XGBoost model
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Defining the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'scale_pos_weight': [scale_pos_weight]  # Setting the calculated scale_pos_weight
}

# Defining GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1',  # Use F1 score as the evaluation metric
    cv=3,  # 3-fold cross-validation
    verbose=3,
    n_jobs=-1
)


grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)

# Evaluating the best model on the validation set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

# Calculating evaluation metrics
f1 = f1_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

best_params = grid_search.best_params_
print("Best Hyperparameters from GridSearchCV:", best_params)

print("\nClassification Report:\n", classification_report(y_val, y_pred))
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux201

[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.122 total time=   4.0s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.144 total time=  11.7s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=1;, score=0.147 total time=   4.8s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=1;, score=0.142 total time=   8.5s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=7, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.177 total time=   7.2s
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=7, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=1;, score=0.173 total tim

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:13:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:13:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:13:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (

[CV 1/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.119 total time=   4.0s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=1;, score=0.145 total time=   6.2s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.135 total time=   5.5s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=1;, score=0.131 total time=   4.8s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=1;, score=0.139 total time=   8.3s
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.01, max_depth=7, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.177 total tim

...

[CV 2/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=50, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.163 total time=   5.2s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.197 total time=   8.5s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=1;, score=0.250 total time=  14.2s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=7, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.308 total time=  12.3s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=7, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.425 total time=  21.1s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=3, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=1;, score=0.190 total time

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:15:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:15:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:15:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (

[CV 2/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=3, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.171 total time=   6.7s
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=3, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.192 total time=  12.3s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=5, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.254 total time=   8.4s
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=5, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=1;, score=0.328 total time=  14.4s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=7, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.412 total time=  11.5s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.2, max_depth=7, n_estimators=200, scale_pos_weight=173.59609895337775, subsample=1;, score=0.540 total tim

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:17:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [05:17:10] WARNING: /workspace/src/lea

Best Parameters: {'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 173.59609895337775, 'subsample': 0.8}
Best Hyperparameters from GridSearchCV: {'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'scale_pos_weight': 173.59609895337775, 'subsample': 0.8}

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    312771
           1       0.41      0.84      0.55      1802

    accuracy                           0.99    314573
   macro avg       0.71      0.92      0.77    314573
weighted avg       1.00      0.99      0.99    314573

F1 Score: 0.5526
Precision: 0.4109
Recall: 0.8435
[CV 1/3] END colsample_bytree=1, learning_rate=0.1, max_depth=3, n_estimators=100, scale_pos_weight=173.59609895337775, subsample=0.8;, score=0.139 total time=   7.0s
[CV 1/3] END colsample_bytree=1, learning_rate=0.1, max_depth=3, n_estimators=200, scale_pos